In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pickle
import statistics

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import nltk
from google.colab import files
import pandas as pd
from datasets import load_metric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -q sentence-transformers
!pip install -q datasets
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.3 MB/s eta 0:00

In [ ]:
rouge = load_metric("rouge")

In [ ]:
print(torch.cuda.is_available())
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print("Using device:", device)

False
Using device: cpu


In [ ]:
from sentence_transformers import SentenceTransformer
sbert = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Preprocessing
df_train = pd.read_json('liscu_train.jsonl', lines=True)
df_test = pd.read_json('liscu_test.jsonl', lines=True)
df_val = pd.read_json('liscu_val.jsonl', lines=True)

In [ ]:
df_train['inputs'] = df_train[['character_name','summary']].agg("</s>".join, axis=1)
df_test['inputs'] = df_test[['character_name','summary']].agg("</s>".join, axis=1)
df_val['inputs'] = df_val[['character_name','summary']].agg("</s>".join, axis=1)

In [ ]:
#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = sbert.encode(sentences)

In [ ]:
from torch.nn.utils.rnn import pad_sequence
a = torch.ones(25, 12)
b = torch.ones(22, 12)
c = torch.ones(15, 12)
cool = pad_sequence([a,b,c])
(cool == 0).sum(dim=2) == cool.size()[-1]

tensor([[False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False, False],
        [False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False, False,  True],
        [False,  True,  True],
        [False,  True,  True],
        [False,  True,  True]])

In [ ]:
from tqdm import tqdm
class Character(Dataset):
    """Single-Turn version of Cornell Movie Dialog Cropus dataset."""

    def __init__(self, inputs, characters, descriptions, device, sbert):
        """
        Args:
            conversations: list of tuple (src_string, tgt_string) 
                         - src_string: String of the source sentence
                         - tgt_string: String of the target sentence
            vocab: Vocabulary object that contains the mapping of 
                    words to indices
            device: cpu or cuda
        """
        self.inputs = inputs
        self.characters = characters
        self.descriptions = descriptions
        self.sbert = sbert
        self.device = device

        # We will pre-tokenize the conversations and save in id lists for later use

        self.tokenized_inputs = []
        for charName, sentences in tqdm(zip(characters, inputs), total=len(characters)):
            mini = sbert.encode([charName]).tolist()
            mini += sbert.encode(sentences).tolist()
            self.tokenized_inputs.append(mini)
        
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        return {"inputs":self.tokenized_inputs[idx], "text":self.descriptions[idx], "character_name": self.characters[idx]}

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (src_seq, trg_seq).
    We should build a custom collate_fn rather than using default collate_fn,
    because merging sequences (including padding) is not supported in default.
    Seqeuences are padded to the maximum length of mini-batch sequences (dynamic padding).
    Args:
        data: list of dicts {"conv_ids":(src_ids, tgt_ids), "conv":(src_str, trg_str)}.
            - src_ids: list of src piece ids; variable length.
            - tgt_ids: list of tgt piece ids; variable length.
            - src_str: String of src
            - tgt_str: String of tgt
    Returns: dict { "conv_ids":     (src_ids, tgt_ids), 
                    "conv":         (src_str, tgt_str), 
                    "conv_tensors": (src_seqs, tgt_seqs)}
            src_seqs: torch tensor of shape (src_padded_length, batch_size).
            tgt_seqs: torch tensor of shape (tgt_padded_length, batch_size).
            src_padded_length = length of the longest src sequence from src_ids
            tgt_padded_length = length of the longest tgt sequence from tgt_ids
    """
    # Sort conv_ids based on decreasing order of the src_lengths.
    # This is required for efficient GPU computations.
    src_ids = [torch.tensor(e["inputs"]) for e in data]
    # src_ids.sort(key=lambda x: len(x), reverse=True)
    descriptions = [e['text'] for e in data]
    
    src_seqs = pad_sequence(src_ids, padding_value=0)
    mask = (src_seqs == 0).sum(dim=2)
    mask = mask != src_seqs.size()[-1]

    return {"inputs": src_ids, "descriptions": descriptions, "inputs_tensors": src_seqs.to(device), 'mask':mask.to(device)}

In [ ]:
df_train.columns

Index(['book_title', 'source', 'character_name', 'summary', 'description',
       'masked_description', 'inputs'],
      dtype='object')

In [ ]:
# Create the DataLoader for all_conversations
train_dataset = Character(df_train['summary'].apply(lambda x: nltk.tokenize.sent_tokenize(x)), 
                          df_train['character_name'], df_train['description'], device, sbert)

  1%|          | 88/7600 [00:55<1:19:39,  1.57it/s]


KeyboardInterrupt: ignored

In [ ]:
train_dataset = torch.load("/content/drive/MyDrive/NLP Final Project/train.pt", map_location=torch.device('cpu'))

In [ ]:
# torch.save(train_dataset, "train.pt")

In [ ]:
# !cp "train.pt" "/content/drive/MyDrive/NLP Final Project/train.pt"

In [ ]:
batch_size = 16

train_data_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, 
                               shuffle=False, collate_fn=collate_fn)

In [ ]:
from torch.distributions.categorical import Categorical
class RLprobs(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        
        self.emb_dim = emb_dim

        encoder_layer = nn.TransformerEncoderLayer(d_model=emb_dim, nhead=8)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)        
        self.linear = nn.Linear(emb_dim, 1)
        self.sigmoid = nn.Sigmoid()

        # self.rouge = evaluate.load('rouge')

    def forward(self, source, mask):
        """
        inputs:
            source - (max_num_sentences, batch_size, emb_dim), where batch_size = number of summaries we are looking at
            mask - (max_num_sentences, batch_size) = for masking the probabilities of padding
        
        
        outputs: 
           probs - (max_num_sentences, batch_size) = p(s_i) should be kept
                (if < 0.5, then remove, else keep)
        """
        encoded = self.transformer_encoder(source)
        probs = self.sigmoid(self.linear(encoded)).squeeze()
        probs[~mask] = 1
        return probs
        

    def train_step(self, probs, target, optimizer, mask, output_scores=False):
        """Run the model on the source and compute the loss on the target.
           The loss for this project should use teacher forcing, where the
           output of the model is used only to compute loss and not passed
           back in to get the next predicted token.

        Args:
            source: An integer tensor with shape (max_source_sequence_length,
                batch_size) containing subword indices for the source sentences.
            target: An integer tensor with shape (max_target_sequence_length,
                batch_size) containing subword indices for the target sentences.

        Returns:
            A scalar float tensor representing cross-entropy loss on the current batch
            divided by the number of target tokens in the batch.
            Many of the target tokens will be pad tokens. You should mask the loss 
            from these tokens using appropriate mask on the target tokens loss.
        """
        probs = torch.cat((probs[:,:,None],1-probs[:,:,None]), dim=2)
        # print(probs)
        category = Categorical(probs)
        # print(category.sample())
        samples = category.sample() * mask
        # print(samples)
        scores = []
        log_prob = category.log_prob(samples)
        for batch_col in range(probs.size()[-2]):
            optimizer.zero_grad()
            policy_loss = 0
            #calculating reward for 1 example in batch
            indices_for_curr_col = (samples[1:,batch_col] == 1).nonzero(as_tuple=True)[0].detach().cpu().numpy()
            summary = " ".join(np.array(target[batch_col])[indices_for_curr_col])
            results = rouge.compute(predictions=[summary], references=[target[batch_col]], rouge_types=['rouge1','rouge2','rougeL'])
            G = np.array([x.mid.fmeasure for x in results.values()]).mean()
            if output_scores:
                scores.append(G)
            ##########################################
            count = 0
            for i, p in enumerate(probs[:, batch_col]):
                if mask[i,batch_col] == True:
                    break
                policy_loss = -log_prob[i,batch_col] * G
                policy_loss.backward()
            optimizer.step()
        if output_scores:
            return scores      
        return []  

In [ ]:
# a = torch.tensor([[0.1,0.2,0.9],
#                   [0.1,0.3,0.4],
#                   [0.1,0.3,0.4],
#                   [0.1,0.3,0.4]])
# probs = torch.cat((a[:,:,None],1-a[:,:,None]), dim=2)
# p = Categorical(probs)
# samples = p.sample()
# p.log_prob(samples)

In [ ]:
rl = RLprobs(384).to(device)
optimizer = torch.optim.AdamW(rl.parameters(), lr=0.01)
rl.load_state_dict(torch.load("/content/drive/MyDrive/NLP Final Project/checkpoint5.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
import tqdm
big = []
for epoch in tqdm.notebook.trange(3, desc="training", unit="epoch"):
    print(f"Epoch: {epoch}")
    i = 0
    the_scores = []
    with tqdm.notebook.tqdm(
                train_data_loader,
                desc="epoch {}".format(epoch + 1),
                unit="batch",
                total=len(train_data_loader)) as batch_iterator:
            rl.train()
            for x, batch_data in enumerate(batch_iterator, start=1):
                i = 0
                forward = rl.forward(batch_data['inputs_tensors'],batch_data['mask'])
                the_scores += rl.train_step(forward, np.array(batch_data['descriptions']), optimizer, batch_data['mask'], output_scores = True)
    torch.save(rl.state_dict(), "/content/drive/MyDrive/NLP Final Project/checkpoint"+str(epoch+3)+".pt")
    big.append(the_scores)
    print(big)

training:   0%|          | 0/3 [00:00<?, ?epoch/s]

Epoch: 0


epoch 1:   0%|          | 0/475 [00:00<?, ?batch/s]

<ipython-input-79-03433c1bf952>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  the_scores += rl.train_step(forward, np.array(batch_data['descriptions']), optimizer, batch_data['mask'], output_scores = True)


[[0.808529913201431, 0.8034044737179761, 0.6793963205016812, 0.7862459954041497, 0.8058226156128253, 0.7781822083560309, 0.6153350734233087, 0.6704620848456465, 0.6988511552162135, 0.762531840286692, 0.6072877185983111, 0.8524468743890806, 0.8985077219886097, 0.9185366253757956, 0.8816166101602261, 0.7109957110786943, 0.9138966760112135, 0.7786146246589792, 0.8264001448829735, 0.7943836895488788, 0.7243211334120425, 0.8554374852263983, 0.9042554611116356, 0.8564463714675264, 0.818663642406621, 0.8916688967387426, 0.848311658185087, 0.8985767615078922, 0.8794664715068367, 0.8172982933417064, 0.869561459906922, 0.7499819637832768, 0.5059266872216512, 0.8754504504504506, 0.7896566092515288, 0.8273089460340456, 0.8701906531764058, 0.8258461190081207, 0.8491440956106455, 0.8251934384395545, 0.743949437179159, 0.6787058790069579, 0.7870383073428758, 0.8242585294179258, 0.8594241901086624, 0.8725405882190799, 0.832005397750443, 0.7254953718368352, 0.8128667202496337, 0.8415749483698972, 0.864

epoch 2:   0%|          | 0/475 [00:00<?, ?batch/s]

[[0.808529913201431, 0.8034044737179761, 0.6793963205016812, 0.7862459954041497, 0.8058226156128253, 0.7781822083560309, 0.6153350734233087, 0.6704620848456465, 0.6988511552162135, 0.762531840286692, 0.6072877185983111, 0.8524468743890806, 0.8985077219886097, 0.9185366253757956, 0.8816166101602261, 0.7109957110786943, 0.9138966760112135, 0.7786146246589792, 0.8264001448829735, 0.7943836895488788, 0.7243211334120425, 0.8554374852263983, 0.9042554611116356, 0.8564463714675264, 0.818663642406621, 0.8916688967387426, 0.848311658185087, 0.8985767615078922, 0.8794664715068367, 0.8172982933417064, 0.869561459906922, 0.7499819637832768, 0.5059266872216512, 0.8754504504504506, 0.7896566092515288, 0.8273089460340456, 0.8701906531764058, 0.8258461190081207, 0.8491440956106455, 0.8251934384395545, 0.743949437179159, 0.6787058790069579, 0.7870383073428758, 0.8242585294179258, 0.8594241901086624, 0.8725405882190799, 0.832005397750443, 0.7254953718368352, 0.8128667202496337, 0.8415749483698972, 0.864

epoch 3:   0%|          | 0/475 [00:00<?, ?batch/s]

[[0.808529913201431, 0.8034044737179761, 0.6793963205016812, 0.7862459954041497, 0.8058226156128253, 0.7781822083560309, 0.6153350734233087, 0.6704620848456465, 0.6988511552162135, 0.762531840286692, 0.6072877185983111, 0.8524468743890806, 0.8985077219886097, 0.9185366253757956, 0.8816166101602261, 0.7109957110786943, 0.9138966760112135, 0.7786146246589792, 0.8264001448829735, 0.7943836895488788, 0.7243211334120425, 0.8554374852263983, 0.9042554611116356, 0.8564463714675264, 0.818663642406621, 0.8916688967387426, 0.848311658185087, 0.8985767615078922, 0.8794664715068367, 0.8172982933417064, 0.869561459906922, 0.7499819637832768, 0.5059266872216512, 0.8754504504504506, 0.7896566092515288, 0.8273089460340456, 0.8701906531764058, 0.8258461190081207, 0.8491440956106455, 0.8251934384395545, 0.743949437179159, 0.6787058790069579, 0.7870383073428758, 0.8242585294179258, 0.8594241901086624, 0.8725405882190799, 0.832005397750443, 0.7254953718368352, 0.8128667202496337, 0.8415749483698972, 0.864

In [ ]:
torch.save(rl.state_dict(), "/content/drive/MyDrive/NLP Final Project/checkpoint3half.pt")

In [ ]:
model_test = RLprobs(384)
model_test.load_state_dict(torch.load("/content/drive/MyDrive/NLP Final Project/checkpoint3half.pt"))

<All keys matched successfully>

In [ ]:
with open("big.npy",'wb') as f:
    np.save(f, big)

In [85]:
f = next(iter(train_data_loader))
prob = rl.forward(f['inputs_tensors'],f['mask'])

In [ ]:
# removed = (prob[:,1] > 0.5).nonzero().numpy().squeeze()
# removed = removed[removed < 37]
# removed

array([ 4,  7,  9, 16, 19, 21, 31])

In [91]:
(prob[:,3] < 0.5).nonzero().numpy().squeeze() < 35

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False])

In [89]:
sum = " ".join(np.array(f['descriptions'][3])[(prob[:,3] < 0.5).nonzero().numpy().squeeze()].squeeze())

IndexError: ignored

In [97]:
for x in range(16):
    print(x)
    ind = (prob[:,x] < 0.5).nonzero().numpy().squeeze()
    ind = ind[ind < len(f['descriptions'][x])]
    sum = " ".join(np.array(f['descriptions'][x])[ind].squeeze())
    print(df_train['character_name'][x])
    print(sum)
    print(df_train.iloc[x]['description'])
    print(rouge.compute(predictions=[sum], references=[df_train.iloc[x]['description']], rouge_types=['rouge1','rouge2','rougeL']))

0
Winston Smith
Winston Smith is a member of the Outer Party. He works in the Records Department in the Ministry of Truth, rewriting and distorting history. To escape Big Brother's tyranny, at least inside his own mind, Winston begins a diary — an act punishable by death. His every move is watched. No place is safe. He also catches the eye of a dark-haired girl from the Fiction Department, whom he believes is his enemy and wants him destroyed. Winston takes pains to meet her, and when they finally do, Julia draws up a complicated plan whereby they can be alone. Alone in the countryside, Winston and Julia make love and begin their allegiance against the Party and Big Brother. Winston is able to secure a room above a shop where he and Julia can go for their romantic trysts. Winston and Julia fall in love, and, while they know that they will someday be caught, they believe that the love and loyalty they feel for each other can never be taken from them, even under the worst circumstances. 

Hank Morgan runs a munitions and machinery factory in 19th-century Connecticut. He's struck by a crowbar during a dispute with the workers, and wakes up thirteen centuries in the past. A knight from King Arthur's Court takes him prisoner, and Arthur orders him executed. Hank escapes by "predicting" an eclipse and claiming that he could blot out the sun if they executed him. Hank becomes the King's prime minister—a.k.a. The Boss—with a reputation for performing miracles. It makes him a number of enemies, notably the wizard Merlin who doesn't like Hank's new status as Arthur's bestie. Hank doesn't do much to discourage Merlin's resentment, and instead responds by destroying Merlin's tower with explosives and a lightning rod. In the process, he begins confiding in a young man named Clarence, who comes to serve as Boy Wonder to his Batman. Soon after, Hank accompanies the Demoiselle Alisande a la Carteloise—thankfully nicknamed Sandy because that's a mouthful of a name—on a quest to save her mistresses from a band of vicious ogres. As it turns out, the mistresses are nothing but pigs whom the Demoiselle believes are enchanted. Hank plays along by buying the pigs and taking them home. Along the way they meet a group of pilgrims journeying to a holy fountain. Their trip turns into a bummer when they find out that the fountain has gone dry. Merlin is already working on restoring it but Hank figures out that it's a simple leak and fixes it easily, bringing the office score to Hank: 2, Merlin: 0. Hank then sets out on a new adventure, disguising himself as a peasant and travelling the country to see how the poor really live. Arthur decides to come with him, which ultimately causes problems since he can't act like a commoner to save his life. A nobleman captures them and plans to sell them into slavery. When Hank escapes, they intend to hang him, the king and their fellow slaves, but at the last minute Launcelot and the knights of the Round Table arrive to rescue them… riding bicycles provided by Clarence. You'd think that Hank would be ready for a rest by then, but no such luck. A knight he insulted, Sir Sagramor, forces him into a duel, along with a passel of his buddies and enchanted weapons and armor from Merlin. Hank defeats them by pulling them out of their saddles with a lasso, then shooting them dead with a pistol when they try to up the ante. Three years pass. Hank marries Sandy and has a baby with her; he also builds some roads, schools, and factories. Unfortunately, he forgets certain particulars of Arthurian literature, which ultimately catch up to him. Arthur is killed after starting a war with Launcelot, and the country revolts—Hank flees with Clarence, hiding in Merlin's cave with a small group of loyal teenage boys. They build fortifications, including land mines, Gatling guns, and electric fences, then battle to victory against an army of 30,000 sent to wipe them out. After the battle, Hank goes out to tend to the wounded, one of whom recognizes him and stabs him in a flagrant display of unsportsmanlike conduct. Clarence and the boys take him back to the cave, but Merlin has a last trick for them, and disguised as a woman, he puts Clarence in an enchanted sleep and leaves him and the others to die in the cave. Back in the present, Hank lies on his deathbed, dreaming about Sandy but lost forever to the ancient world he helped create.
